# Results tables

A (relatively) simple notebook to generate tables for reporting out in our paper.

##### Import Libraries and Packages

In [1]:
from PIL import Image
import numpy as np
import io
import boto3
import re
import os
import seaborn as sns
import random
from pathlib import Path
from glob import glob
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline

pd.set_option('display.max_rows', 500)

In [2]:
search_path = "./"
file_extension = "*.csv"
os.chdir('/home/ec2-user/SageMaker/w210-capstone/model_results/')
dfs = []
[dfs.append(pd.read_csv(file, index_col=0))
 for path, subdir, files in os.walk(search_path)
 for file in glob(os.path.join(path, file_extension))]

df_orig = pd.concat(dfs, ignore_index=True)
#df_orig['Accuracy'] = df_orig['Accuracy'].apply(lambda x: x*100)
#df_orig['Original_CI'] = df_orig.Original_CI.apply(lambda x: literal_eval(x))
#df_orig['Orig_CI_Low'], df_orig['Orig_CI_Hi'] = zip(*df_orig.Original_CI)

# Filter data into subsets based on test dataset used
df_orig_cifar10 = df_orig[df_orig['Testset'] == 'cifar10']
df_orig_cifar101 = df_orig[df_orig['Testset'] == 'cifar10.1']

In [3]:
final_score_subset = df_orig[['Model', 'Accuracy', 'Testset']].sort_values(by=['Model'])

# Remove models that we did not run randaugment datasets through
final_score_subset = final_score_subset[~final_score_subset["Model"].str.contains('resnet_basic_44')]
final_score_subset = final_score_subset[~final_score_subset["Model"].str.contains('pyramidnet_basic_110_84')]
final_score_subset = final_score_subset[~final_score_subset["Model"].str.contains('resnext_29_8x64d')]


p2 = re.compile("([\d]+)(_refined)?(\d*)")

final_score_subset['ra'] = final_score_subset['Model'].str.contains("ra_")
final_score_subset['cm'] = final_score_subset["Model"].str.contains("cm_")

# make text field for classifying base, cm, or ra
final_score_subset['base'] = np.logical_and(np.logical_not(final_score_subset['ra']), \
                                            np.logical_not(final_score_subset['ra']))

def training_type_extraction(row):
    if row['ra']:
        ret_val = 'RandAugment'
    elif row['cm']:
        ret_val = 'CutMix'
    elif row['base']:
        ret_val = 'Base Model'
    else:
        ret_val = 'Other'
    return ret_val

final_score_subset['Training Type'] = final_score_subset.apply(lambda row: training_type_extraction(row), axis=1)


def ra_N_extraction(row):
    p3 = re.compile("ra_(\d+)_(\d+)")
    N=0 
    if row['ra'] == True :
        srch3 = p3.search(row['Model'].lower())
        N = int(srch3.group(1))
    return N

def ra_M_extraction(row):
    p3 = re.compile("ra_(\d+)_(\d+)")
    M=0 
    if row['ra'] == True :
        srch3 = p3.search(row['Model'].lower())
        M = int(srch3.group(2))
    return M

def cm_alpha_extraction(row):
    p3 = re.compile("cm_(\d+)_([\.\d]+)")
    M=0 
    if row['cm'] == True :
        srch3 = p3.search(row['Model'].lower())
        M = srch3.group(2)
    return M

final_score_subset['N'] = final_score_subset.apply(lambda row: ra_N_extraction(row), axis=1)
final_score_subset['M'] = final_score_subset.apply(lambda row: ra_M_extraction(row), axis=1)
final_score_subset['alpha'] = final_score_subset.apply(lambda row: cm_alpha_extraction(row), axis=1)
cols = ['N', 'M']
final_score_subset['hue'] = final_score_subset[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)


#final_score_subset[final_score_subset['Model'].str.contains("refined300")]
final_score_subset

,Model,Accuracy,Testset,ra,cm,base,Training Type,N,M,alpha,hue
34,densenet_BC_100_12,0.8875,cifar10,False,False,True,Base Model,0,0,0,0_0
37,densenet_BC_100_12,0.8830,cifar10.1,False,False,True,Base Model,0,0,0,0_0
36,densenet_BC_100_12,0.9484,cifar10,False,False,True,Base Model,0,0,0,0_0
35,densenet_BC_100_12,0.9456,cifar10,False,False,True,Base Model,0,0,0,0_0
185,densenet_BC_100_12_cm_1_.5,0.2710,cifar10.1,False,True,True,CutMix,0,0,.5,0_0
184,densenet_BC_100_12_cm_1_.5,0.3093,cifar10,False,True,True,CutMix,0,0,.5,0_0
187,densenet_BC_100_12_cm_1_.5_refined400,0.9374,cifar10,False,True,True,CutMix,0,0,.5,0_0
186,densenet_BC_100_12_cm_1_.5_refined400,0.8630,cifar10.1,False,True,True,CutMix,0,0,.5,0_0
17,densenet_BC_100_12_cm_1_1,0.8250,cifar10.1,False,True,True,CutMix,0,0,1,0_0
16,densenet_BC_100_12_cm_1_1,0.9055,cifar10,False,True,True,CutMix,0,0,1,0_0
